# _Manifest File Based_


### 1. What is Manifest File in Android Apps ?
        Manifest file is Like an Identification for a application just like a Id required for humans. It is basically a declaration sheet that tells the
        program important details about the Application, package etc, so it knows how to handle them.

        It contains :
        1. App Meta Data (name, icons, version, min Sdk, etc...)
        2. Permissions (all the permissions are declared here)
        3. Activities (all the activity information must be present in the manifest file)
        4. Services, Stattic BroadCasts and other Configurations...


### 2. Can there be multiple Manifest File in a single App ?
        Yes, there can be multiple manifest file in a single application. Each Module has its own manifest and so does the library dependencies.
        When we build, gradle MERGES all the manifest files according to some MERGE RULES.

# **Activity Based**

### What is an ACTIVITY in Android ?
     It is a focussed Screen in Apps, where the user can INTERACT WITH the elements of UI like the buttons, card, lists etc.
     It is basically an Entry Point of user interaction.

### What all does Component ACTIVITY do?
     1. Manages the UI components and responds to the User Interaction.
     2. Has its own LIFECYCLE METHODS
     3. Implements the following interfaces : Lifecycle, viewmodel, SavedStateHandle.
     4. Compose Ready (not tied to compose, only compose ready)

    Without it, we would have to manually implement the interfaces it implements like VM, savedStateHandle, lifecycle methods.

### What is the Activity Architecture we follow in Modern Android Development ?
    In Modern Android Dev, we follow a Single Activity Architecture. We have different Screens implemented as COMPOSABLES.

    Benefits :
    1. Easy Navigation Handling (don't have to manage each multiple activity back stacks)
    2. Shared ViewModels (Easy to Manage and share states as a ViewModel is tied to a ViewModelStoreOwner (usually an Activity))
    3. In multi-activity architecture, when you navigate to another activity, the old activity (and its ViewModel) gets destroyed unless you
        explicitly persist data somewhere else.
    4. No need to pass data between acivities using Intents.

### What is the Concept of Layout Inflation (like in XML) in Compose ?
    1. The set content {}  present un MainActivity takes a composable as the root composable
    2. It creates a ROOT COMPOSABLE inside the activity
    3. hooks the composable lifecycle to Activity Cycle
    4. Manages Recomposition when State Changes.

# **Intent Related**

### What is an Intent ?
    Intents are MESSAGING objects, which are used to communicate with another App Components.

    USES -
    1. Start An Activity
    2. Run a service
    3. Deliver a broadcast

### What are the types of Intent ?
     A. Explicit -
        Used to communicate within the same app, example : To Move between different activties or services
     B. Implicit -
        Declares a general action to perform & Android Decides which Component can handle it (shows options when you click on pdf, diff apps are shown)

     Other Types -
     1. BroadCast Intent : Used to deliver system wide app announcements
     2. Pending Intent : For future Work, like alarms, scheduled notification

# **Services Related**

### What are services ?
    Taskes that run in tne Background to perform Long Running Operations without the UI. Useful for taks which should continue even if the app is closed
    or user does not interact with App.

    Examples : Background Music Play, Downlaoding something, Syncing contacts with the server

### What are the types of services ?
    A. ForeGround :
        Runs in foreground with persistent notification
        ex : Downlaod Manager Showing Progess

    B. BackGround :
        Runs Until stopped
        ex : Music Playback in background.

# **BroadCast related**

### What is a broadcast ?
    It is a system wide message that any apps or components can send or receive about an event or trigger.

### What is a broadcast Receiver?
    Android Component that allows an app to lsiten to system wide events without needing to run in background.
    BoradCast recv listens for any such messages runs a code when it comes.

    example : Battery Low, Connectivty Change, On boot Complete.


### What are the types of BroadCast Receivers ?
    A. Static :
        Declared Inside Manifest. ex : On boot completed.
    B. Dynamic :
        Written inside code. ex : Connectivity Change.

# Content Provider Related

### What is Content Provider ?
    Android Component that manages & shares structured data B/W apps. Acts like a middleMan.

    Uses A URI(Uniform Resource Id.)
    Access cam be restricted using Permissions

    Example :
        The contact App provides Contact data through Contacs Contract Provider. Whatsapp can query it to get the comtacts ONLY IF
        it has the required PERMISSIONS.